In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [204]:
query_all_laps=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT * FROM `perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
track_all_laps=pandas_gbq.read_gbq(query_all_laps,project_id,dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning:

A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.



In [205]:
race="Hungarian Grand Prix"

In [206]:
track2=track3[(track3["GP"]==race)&(track3["Year"]==2024)]

In [207]:
#Get Final Pos to order the final plot
track_all_laps_2=track_all_laps[(track_all_laps["GP"]==race)&(track_all_laps["Year"]==2024)]
final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]

In [208]:
final_pos

,Driver,Position
12133,ZHO,19.0
12134,ALB,14.0
12135,NOR,2.0
12136,PIA,1.0
12137,RUS,8.0
12138,TSU,9.0
12139,SAR,17.0
12140,HUL,13.0
12141,SAI,6.0
12142,OCO,18.0


In [209]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]
Drivers=(track2['Driver'].unique())

In [210]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,SOFT,6
1,ALB,2.0,HARD,23
2,ALB,3.0,HARD,40
3,ALO,1.0,SOFT,7
4,ALO,2.0,MEDIUM,30


In [211]:
stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position'])

In [212]:
fig=px.bar(
    stint_with_pos,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="simple_white",
    title="<b>Tyre Strategies for the {} {}</b>".format(year,gp),
    height=800, width=1200,
    hover_data=["Position"],
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="Black",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    margin=dict(l=5, r=5, t=35, b=5),
    )

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [213]:
fig.write_html("{} {} Tyre Strategies.html".format(year,gp))